In [85]:
import pandas as pd
from functions.word_preprocessing import *
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import pickle
import datetime
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
import csv
import numpy as np
import math

pd.set_option('display.max_colwidth', 500)

In [2]:
directory = '/Users/kwheatley/Desktop/Capstone/gcloud_data/'

In [35]:
# Load education for resumes
resume_edu = pd.read_csv(directory+'00_resumes_education.csv',header=None)
resume_edu.columns = ['city','resume_id','container_type','edu_title',
                      'edu_school','edu_addressLocality','edu_dates']

# Remove any null education titles
resume_edu = resume_edu[~resume_edu.edu_title.isnull()]

# Load the list of relevant resume ids from 03_create_ngram_model
relevant_resume_ids = pd.read_csv(directory+'temp_files/relevant_resume_ids.csv')

# Remove all education not in resume id list
resume_edu = resume_edu[resume_edu.resume_id\
                       .isin(relevant_resume_ids['0'])]

print("Number of records:", resume_edu.resume_id.count())
print("Number of unique resume ids:", resume_edu.resume_id.nunique())

Number of records: 178403
Number of unique resume ids: 102443


In [37]:
# Clean education titles and tokenize
edu_titles = data.edu_title
edu_titles = [re.sub('[^A-Za-z0-9\s]+', '', row.lower()) for row in edu_titles]
edu_tokens = [nltk.word_tokenize(row) for row in edu_titles]

In [69]:
acronym_list = []
for row in edu_tokens:
    [acronym_list.append(x) for x in row if len(x) < 5]
acronym_list = list(set(acronym_list))

degree_type_dict = pd.read_csv('functions/configuration_files/degree_type_dict.csv', encoding='latin-1')
degree_type_dict = degree_type_dict[['keyword','type']]\
                        .set_index('keyword')['type']\
                        .to_dict()

degree_name_list = []
subject_name_list = []
for row in edu_tokens:
    
    degree_row = []
    subject_row = row
    for token in row:
        if token == 'in':
            degree_row.append(token)
            subject_row = subject_row[1:]
            break
        elif token in list(degree_type_dict.keys()) + acronym_list:
            degree_row.append(token)
            subject_row = subject_row[1:]
        else:
            break

    degree_name_list.append(' '.join(degree_row))
    subject_name_list.append(' '.join(subject_row))

In [38]:
# for row in edu_titles:
#     if 'aa' in row:
#         print(row)
#         print()

In [92]:
degree_type_dict = pd.read_csv('functions/configuration_files/degree_type_dict.csv', encoding='latin-1')
degree_type_dict = degree_type_dict[['keyword','type']]\
                        .set_index('keyword')['type']\
                        .to_dict()


In [93]:
degree_category_list = []
for row in degree_name_list:
    degree_category = []
    for key in degree_type_dict:
        if key in row.split():
            degree_category.append(degree_type_dict[key])
    degree_category_list.append(' '.join(set(degree_category)))

final = list(zip(edu_titles, degree_name_list, subject_name_list,degree_category_list))
pd.DataFrame(list(set(final))).to_csv('final.csv')
pd.DataFrame(list(set(zip(degree_name_list, degree_category_list)))).to_csv('degree_name_list.csv')

In [68]:
counter_temp = Counter(degree_name_list)    
writefile = open('test1.csv', 'w')
writer = csv.writer(writefile)
for key, count in counter_temp.items():
    writer.writerow([str(key),str(count)])
writefile.close()

pd.DataFrame(list(set(subject_name_list))).to_csv('temp.csv')

In [36]:
data.loc[data.edu_dates.isnull(),'edu_dates'] = 'January 1900'
data['from_date'] = data.edu_dates.apply(lambda x: (x.split(" to ")[0]).strip())
data['from_date'] = data.from_date.apply(lambda x: 
                        datetime.datetime.strptime('January ' + x, '%B %Y')
                        if len(x.split(" "))==1
                        else datetime.datetime.strptime(x, '%B %Y'))
data['to_date'] = data.edu_dates.apply(lambda x: (x.split(" to ")[1]).strip()
                        if len(x.split(" to "))>1
                        else x.strip())
data['currently_here'] = 'No'
data.loc[data.to_date == 'Present','currently_here'] = 'Yes'
data.loc[data.to_date == 'Present','to_date'] = 'July 2018'
data['to_date'] = data.to_date.apply(lambda x: 
                        datetime.datetime.strptime('December ' + x, '%B %Y')
                        if len(x.split(" "))==1
                        else datetime.datetime.strptime(x, '%B %Y'))
data['from_year'] = data.from_date.apply(lambda x: str(x.year))
data['to_year'] = data.to_date.apply(lambda x: str(x.year))
data['days_edu'] = (data.to_date - data.from_date).dt.days
data['years_edu'] = round(data.days_edu/365)

In [ ]:
# ngram_counter = Counter()

for row in edu_tokens:
    unigrams = ngrams(row,1)
#     bigrams = ngrams(row,2)
#     tigrams = ngrams(row,3)

    ngram_counter.update(unigrams)
#     ngram_counter.update(bigrams)
#     ngram_counter.update(tigrams) 

In [11]:
# for i in resume_work.groupby(['role','cleaned_job_title','experiences','currently_here'])\
# ['days_worked'].agg('mean').reset_index()['experiences'].unique():
#     print(i)
# resume_work[resume_work.experiences == '4, 7'].role

In [ ]:
# https://developers.google.com/chart/interactive/docs/gallery/sankey